# Hard and soft constraints

### Hard constraints
These are constraints that must be pursued during the search and without meeting these constraints, the solution is not true. These constraints are:
- One teacher cannot give two courses at the same time
- A set of students cannot have two classes at the same time
- A teacher cannot teach more than maximum allotted hours.
- Classes Cannot be on off days (Saturday and Sunday)
- Maximum number of hours allotted for a day cannot be exceeded.

### Soft constraints
These constraints that are not mandatory for a solution must be pursued, but the quality of the courses schedule is decided by following these constraints
- In the timetable of teacher, no more than two lectures a day, each one 1 hours and 30 minutes, breaks are preferred 30 minutes or more.
- In the students' timetable, we have to put three lectures a day at maximum with 30 minutes breaks between lectures so that they have more time for self-study at home
- We must ensure that a group of students does not have to attend school only to attend one class, there has to be a maximum of three classes in a day and a minimum of two classes so that it will be worth it to come to college on a particular day

# Initialization

In [8]:
# Student groups:
stg_0 = [0,0]
stg_1 = [0,1]
stg_2 = [1,0]
stg_3 = [1,1]

# Student Groups Weekly hours -> will be defined later, each student group will have separate number of weekly hours 10-15

# List of courses:
course_0 =  [0,0,0,0]
course_1 = [0,0,0,1]
course_2 = [0,0,1,0]
course_3 = [0,0,1,1]

# Courses weekly hours -> each course can have multiple hours per week
# Professor for which course -> each course can have one or more professors available to teach

# List of professors:
prof_0 = [0,0,0,0]
prof_1 = [0,0,0,1]
prof_2 = [0,0,1,0]
prof_3 = [0,0,1,1]

# Professor availability -> each professor has availability only for certain day and time

# Time slots availability -> part of constraints [defined]
## Days (monday-friday)
day_0 = [0, 0, 0]
day_1 = [0, 0, 1]
day_2 = [0, 1, 0]
day_3 = [0, 1, 1]
day_4 = [1, 0, 0]
## lecture time
time_0 = [0, 0, 0] # 08.30 - 10.00
time_1 = [0, 0, 1] # 10.30 - 12.00
time_2 = [0, 1, 0] # 12.30 - 14.00

# Evolutionary steps

In [9]:
# Initial population
time_table_0 = stg_0 + course_0 + prof_0 + day_0 + time_0
time_table_1 = stg_0 + course_1 + prof_1 + day_0 + time_1
time_table_2 = stg_0 + course_2 + prof_2 + day_0 + time_2

In [10]:
print(time_table_0)
print(time_table_1)
print(time_table_2)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]
